# Modularizing the Testbench

In [1]:
from pyuvm import *
from tb_pkg import *
import random

class AluDut(uvm_component):
    """Provides the alu TLM operations"""
    @staticmethod
    def alu_op(A, B, op, is_dut = False):
        """TLM model of the TinyALU"""
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        if is_dut:
            time.sleep(0.1)
            if random.randint(0,3) == 0:
                result = result + 1
        return result

class Coverage(uvm_component):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, op):
        assert isinstance(op, Ops), "Coverage can only receive Ops"
        self.cvg.add(op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")

class Scoreboard(uvm_component):  

    def end_of_elaboration_phase(self):
        self.results=[]

    def write(self, op_result):
        A, B, op, actual_result = op_result
        predicted_result = AluDut.alu_op(A, B, op)
        self.results.append( (A, B, op, predicted_result, actual_result) )

    def check_phase(self):
        for A, B, op, predicted_result, actual_result in self.results:
            if predicted_result == actual_result:
                self.logger.info( f"PASSED: {A:02x} {op.name} {B:02x} = {actual_result:04x}")
            else:
                self.logger.error(f"FAILED: {A:02x} {op.name} {B:02x} = {actual_result:04x} expected {predicted_result:04x}")


class AluEnv(uvm_env):
    def build_phase(self):
        self.cvg = Coverage("cvg", self)
        self.scb = Scoreboard("scb", self)
        self.dut = AluDut("dut", self)
    
    def run_phase(self):
        self.raise_objection()  ## You MUST raise an objection

        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            self.cvg.write(op)
            actual_result = self.dut.alu_op(A, B, op, is_dut=True)
            result_tuple = ( A, B, op, actual_result)
            self.scb.write( result_tuple )
        self.drop_objection()  ## drop the objection to end

class AluTest(uvm_test):
    def build_phase(self):
        self.env = AluEnv("env", self)

uvm_root().run_test("AluTest")

ERROR: <ipython-input-1-ed280dd272b4>(36)[uvm_test_top.env.cvg]: Functional coverage error. Missed: {<Ops.MUL: 4>}
INFO: <ipython-input-1-ed280dd272b4>(51)[uvm_test_top.env.scb]: PASSED: cb AND 6e = 004a
INFO: <ipython-input-1-ed280dd272b4>(51)[uvm_test_top.env.scb]: PASSED: cf XOR 8b = 0044
INFO: <ipython-input-1-ed280dd272b4>(51)[uvm_test_top.env.scb]: PASSED: 38 ADD 6d = 00a5
INFO: <ipython-input-1-ed280dd272b4>(51)[uvm_test_top.env.scb]: PASSED: 26 XOR 0f = 0029
ERROR: <ipython-input-1-ed280dd272b4>(53)[uvm_test_top.env.scb]: FAILED: fb XOR 1b = 00e1 expected 00e0
